In [ ]:
from sunflower.config import *
from sunflower.core import *
from sunflower.test import *

In [ ]:
payslips=Raw(user['stephen']['raw'],payslip['files'])


In [ ]:
raw_payslips=payslips.read_files(10)

In [ ]:
a=RawTest(raw_payslips).column_test()

In [ ]:
df = pd.DataFrame.from_dict(a,orient='index')
df=df.transpose()


In [ ]:
a.to_csv(user['stephen']['test']+"column.csv")

In [ ]:
a=set([1,2,3])
b=set([1,3,2])
set.intersection(*[a,b])

In [ ]:
class RawTest:
    def __init__(self,sample:dict):
        self.sample=sample
    
    def sheet_test(self): #milti-sheets detect
        multisheets={}
        for i in self.sample:
            if isinstance(self.sample[i], dict):
                identical=all([set(self.sample[i][list(self.sample[i].keys())[0]].columns) == set(self.sample[i][df].columns) for df in self.sample[i]])
                multisheets[i]=identical
        return multisheets

    def column_test(self): #df detect
        columns={}
        benchmark=list(self.sample[list(self.sample.keys())[0]].columns)
        columns[','.join([str(element) for element in benchmark])]=[list(self.sample.keys())[0]]
        for i in self.sample:
            if isinstance(self.sample[i], dict)==False:
                if list(self.sample[i].columns)!=benchmark:
                    columns[','.join([str(element) for element in list(self.sample[i].columns)])]=[i]
                    benckmark=list(self.sample[i].columns)
                else:
                    columns[','.join([str(element) for element in benchmark])].append(i)
        col=[set(list(i.split(","))) for i in list(columns.keys())]
        coldiff=set.union(*col) - set.intersection(*col)
        for i in list(columns.keys()):
            result=set.intersection(*[set(list(i.split(","))),coldiff])
            columns[i].append(list(result))
        return columns

In [ ]:
import pandas as pd
import datetime,os
import numpy as np

class RawTest:
    def __init__(self,sample:dict):
        self.sample=sample
    
    def sheet_test(self): #milti-sheets detect
        multisheets={}
        for i in self.sample:
            if isinstance(self.sample[i], dict):
                identical=all([set(self.sample[i][list(self.sample[i].keys())[0]].columns) == set(self.sample[i][df].columns) for df in self.sample[i]])
                multisheets[i]=identical
        return multisheets

    def column_test(self): #df detect
        file_col=[]
        commoncol=set.intersection(*[set(list(self.sample[i].columns)) for i in self.sample.keys()])
        cat=0
        for i in self.sample:
            if isinstance(self.sample[i], dict)==False:
                extra=set.union(*[set(list(self.sample[i].columns)),commoncol])-set.intersection(*[set(list(self.sample[i].columns)),commoncol])
                file_col.append([i,list(commoncol),list(self.sample[i].columns),list(extra)])
        test_result=pd.DataFrame(
                                columns=['Filename','Common Col','Files Columns','Extra Columns'],
                                data=file_col
                                )
        return test_result
        

In [ ]:
a=RawTest(raw_payslips).sheet_test()

In [ ]:
raw_payslips['test']={
        'sheets1':pd.DataFrame({
            'col1':[1,2,3,4],
            'col2':[1,2,3,4],
            'col3':[1,2,3,4],
            'col4':[1,2,3,4],
        }),
        'sheets2':pd.DataFrame({
            'col1':[1,2,3,4],
            'col2':[1,2,3,4],
            'col3':[1,2,3,4],
            'col4':[1,2,3,4],
            'col5':[1,2,3,4],
        }),
        'sheets3':pd.DataFrame({
            'xxx':[1,2,3,4],
            'col2':[1,2,3,4],
            'col3':[1,2,3,4],
            'col4':[1,2,3,4],
        })
    }

raw_payslips['test2']={
        'sheets3':pd.DataFrame({
            'yy':[1,2,3,4],
            'colss2':[1,2,3,4],
            'col3':[1,2,3,4],
            'col4':[1,2,3,4],
        }),
        'sheets4':pd.DataFrame({
            'col1':[1,2,3,4],
            'colss2':[1,2,3,4],
            'col3':[1,2,3,4],
            'col4':[1,2,3,4],
            'col5':[1,2,3,4],
        }),
        'sheets5':pd.DataFrame({
            'xxx':[1,2,3,4],
            'colss2':[1,2,3,4],
            'col3':[1,2,3,4],
            'col4':[1,2,3,4],
        })
    }


    

In [ ]:
b= pd.concat([raw_payslips['test'][i] for i in raw_payslips['test']],ignore_index=True, sort=False)

In [ ]:
b

In [ ]:
import pandas as pd
import datetime,os
import numpy as np
import copy

class RawTest:
    def __init__(self,sample:dict):
        self.sample=sample
    
    def sheet_test(self): #milti-sheets detect
        result=[]
        for i in self.sample:
            if isinstance(self.sample[i], dict):
                allcol=set.union(*[set(list(self.sample[i][j].columns)) for j in self.sample[i].keys()])
                comcol=set.intersection(*[set(list(self.sample[i][j].columns)) for j in self.sample[i].keys()])
                if allcol!=comcol:
                    test=self.column_test(self.sample[i])
                    test['File']=i
                    result.append(test)
        result = pd.concat(result)
        return result

    def column_test(self,sample): #df detect
        sample_dict=copy.deepcopy(sample)
        dict_instance=[]
        for i in sample_dict:
            if isinstance(sample_dict[i], dict):
                dict_instance.append(i)
        for i in dict_instance:
            sample_dict.pop(i,None)      
        file_col=[]
        commoncol=set.intersection(*[set(list(sample_dict[i].columns)) for i in sample_dict.keys()])
        cat=0
        for i in sample_dict:
            if isinstance(sample_dict[i], dict)==False:
                extra=set.union(*[set(list(sample_dict[i].columns)),commoncol])-set.intersection(*[set(list(sample_dict[i].columns)),commoncol])
                file_col.append([i,list(commoncol),list(sample_dict[i].columns),list(extra)])
        test_result=pd.DataFrame(
                                columns=['Filename','Common Col','Files Columns','Extra Columns'],
                                data=file_col
                                )
        return test_result
        
        

        

In [ ]:
a=RawTest(raw_payslips).sheet_test()

In [ ]:
raw_payslips['test2']

In [ ]:
a